In [ ]:
from numpy.ma.core import sqrt
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats.mstats_basic import describe
import os
import glob
from os.path import basename
from functools import reduce


pd.set_option('max_row', 99999)
pd.set_option('max_colwidth', 400)
pd.set_option('display.max_columns', None) #display all columns from a dataframe

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

#Loading participants' answers to the entry survey
df_Entry = pd.read_excel ('TestData.xlsx', sheet_name='Sheet1')
df_Entry = df_Entry.fillna('')
display(df_Entry)

,ParticipantID,Entery_1_Predict,Entery_1_Reason_Opt1,Entery_1_Reason_Opt1_6_TEXT,Entery_1_Reason_Opt2,Entery_1_Reason_Opt2_6_TEXT,Entery_1_Reason_Opt3,Entery_1_Reason_Opt3_6_TEXT,Entery_1_Reason_Opt4,Entery_1_Reason_Opt4_6_TEXT,Entery_1_Reason_Opt5,Entery_1_Reason_Opt5_6_TEXT,Entery_1_Fix_Opt1,Entery_1_Fix_Opt1_7_TEXT,Entery_1_Fix_Opt2,Entery_1_Fix_Opt2_7_TEXT,Entery_1_Fix_Opt3,Entery_1_Fix_Opt3_7_TEXT,Entery_1_Fix_Opt4,Entery_1_Fix_Opt4_7_TEXT,Entery_1_Fix_Opt5,Entery_1_Fix_Opt5_7_TEXT,Entery_2_Predict,Entery_2_Reason_Opt1,Entery_2_Reason_Opt1_6_TEXT,Entery_2_Reason_Opt2,Entery_2_Reason_Opt2_6_TEXT,Entery_2_Reason_Opt3,Entery_2_Reason_Opt3_6_TEXT,Entery_2_Reason_Opt4,Entery_2_Reason_Opt4_6_TEXT,Entery_2_Reason_Opt5,Entery_2_Reason_Opt5_6_TEXT,Entery_2_Fix_Opt1,Entery_2_Fix_Opt1_7_TEXT,Entery_2_Fix_Opt2,Entery_2_Fix_Opt2_7_TEXT,Entery_2_Fix_Opt3,Entery_2_Fix_Opt3_7_TEXT,Entery_2_Fix_Opt4,Entery_2_Fix_Opt4_7_TEXT,Entery_2_Fix_Opt5,Entery_2_Fix_Opt5_7_TEXT,Entery_3_Predict,Entery_3_Reason_Opt1,Entery_3_Reason_Opt1_6_TEXT,Entery_3_Reason_Opt2,Entery_3_Reason_Opt2_6_TEXT,Entery_3_Reason_Opt3,Entery_3_Reason_Opt3_6_TEXT,Entery_3_Reason_Opt4,Entery_3_Reason_Opt4_6_TEXT,Entery_3_Reason_Opt5,Entery_3_Reason_Opt5_6_TEXT,Entery_3_Fix_Opt1,Entery_3_Fix_Opt1_7_TEXT,Entery_3_Fix_Opt2,Entery_3_Fix_Opt2_7_TEXT,Entery_3_Fix_Opt3,Entery_3_Fix_Opt3_7_TEXT,Entery_3_Fix_Opt4,Entery_3_Fix_Opt4_7_TEXT,Entery_3_Fix_Opt5,Entery_3_Fix_Opt5_7_TEXT,Entery_4_Predict,Entery_4_Reason_Opt1,Entery_4_Reason_Opt1_6_TEXT,Entery_4_Reason_Opt2,Entery_4_Reason_Opt2_6_TEXT,Entery_4_Reason_Opt3,Entery_4_Reason_Opt3_6_TEXT,Entery_4_Reason_Opt4,Entery_4_Reason_Opt4_6_TEXT,Entery_4_Reason_Opt5,Entery_4_Reason_Opt5_6_TEXT,Entery_4_Fix_Opt1,Entery_4_Fix_Opt1_7_TEXT,Entery_4_Fix_Opt2,Entery_4_Fix_Opt2_7_TEXT,Entery_4_Fix_Opt3,Entery_4_Fix_Opt3_7_TEXT,Entery_4_Fix_Opt4,Entery_4_Fix_Opt4_7_TEXT,Entery_4_Fix_Opt5,Entery_4_Fix_Opt5_7_TEXT,Entery_5_Predict,Entery_5_Reason_Opt1,Entery_5_Reason_Opt1_6_TEXT,Entery_5_Reason_Opt2,Entery_5_Reason_Opt2_6_TEXT,Entery_5_Reason_Opt3,Entery_5_Reason_Opt3_6_TEXT,Entery_5_Reason_Opt4,Entery_5_Reason_Opt4_6_TEXT,Entery_5_Reason_Opt5,Entery_5_Reason_Opt5_6_TEXT,Entery_5_Fix_Opt1,Entery_5_Fix_Opt1_7_TEXT,Entery_5_Fix_Opt2,Entery_5_Fix_Opt2_7_TEXT,Entery_5_Fix_Opt3,Entery_5_Fix_Opt3_7_TEXT,Entery_5_Fix_Opt4,Entery_5_Fix_Opt4_7_TEXT,Entery_5_Fix_Opt5,Entery_5_Fix_Opt5_7_TEXT
0,O_P9,"1,3",3,,,,5,,,,,,3.0,,,,6.0,,,,,,1,2,,,,,,,,,,2,,,,,,,,,,"1,3",3,,,,4,,,,,,4.0,,,,3,,,,,,"3,5",,,,,3.0,,,,5,,,,,,4.0,,,,6.0,,"1,3",1.0,,,,2,,,,,,1.0,,,,4.0,,,,,
1,O_P13,"1,3,5",4,,,,5,,,,3,,3.0,,,,6.0,,,,4.0,,"1,3,4,5",2,,,,1,,3.0,,3.0,,2,,,,1.0,,4.0,,4.0,,"1,3,4,5",3,,,,4,,3.0,,3.0,,4.0,,,,3,,4.0,,4.0,,"1,3,5",5,,,,3.0,,,,3,,6.0,,,,4.0,,,,4.0,,"1,3",1.0,,,,2,,,,,,1.0,,,,2.0,,,,,
2,O_P14,"1,3","3,4",,,,"3,5",,,,,,3.0,,,,6.0,,,,,,"1,2,3","2,3",,6.0,,"1,3",,,,,,2,,4.0,,1.0,,,,,,"1,3,4",3,,,,"1,4",,3.0,,,,4.0,,,,1,,7.0,,,,"1,3","4,5",,,,3.0,,,,,,6.0,,,,4.0,,,,,,"1,3,5",1.0,,,,"2,3",,,,6.0,,1.0,,,,2.0,,,,4.0,
3,O_P17,"1,3",4,,,,"1,5",,,,,,5.0,,,,6.0,,,,,,1,2,,,,,,,,,,2,,,,,,,,,,3,,,,,"4,5",,,,,,,,,,3,,,,,,"1,5","4,5",,,,,,,,"1,5",,6.0,,,,,,,,6.0,,"1,3",1.0,,,,2,,,,,,1.0,,,,2.0,,,,,
4,O_P18,"1,3",4,,,,5,,,,,,3.0,,,,6.0,,,,,,1,2,,,,,,,,,,2,,,,,,,,,,"1,3","3,4",,,,4,,,,,,4.0,,,,4,,,,,,3,,,,,3.0,,,,,,,,,,5.0,,,,,,"1,3",1.0,,,,2,,,,,,1.0,,,,2.0,,,,,
5,I_P12,1,"3,4",,,,,,,,,,3.0,,,,,,,,,,1,2,,,,,,,,,,2,,,,,,,,,,"1,3",3,,,,"3,4",,,,,,5.0,,,,5,,,,,,1,5,,,,,,,,,,6.0,,,,,,,,,,"1,3",1.0,,,,"2,5",,,,,,1.0,,,,2.0,,,,,
6,I_P13,"3,5",,,,,5,,,,"1,3",,,,,,7.0,,,,4.0,,"1,2,3","1,2",,6.0,,1,,,,,,2,,7.0,,1.0,,,,,,"1,3,4",3,,,,4,,6.0,,,,4.0,,,,4,,7.0,,,,"1,3,5",4,,,,3.0,,,,"1,3",,6.0,,,,4.0,,,,2.0,,"1,3,5",1.0,,,,2,,,,6.0,,1.0,,,,2.0,,,,7.0,
7,I_P14,"1,3,5",4,,,,"1,3,5",,,,"1,3",,3.0,,,,6.0,,,,4.0,,"1,3",2,,,,1,,,,,,2,,,,5.0,,,,,,3,,,,,4,,,,,,,,,,5,,,,,,"1,5",5,,,,,,,,1,,6.0,,,,,,,,4.0,,"1,3",1.0,,,,2,,,,,,1.0,,,,2.0,,,,,
8,O_P19,"1,3",4,,,,5,,,,,,3.0,,,,6.0,,,,,,1,"1,2,3",,,,,,,,,,2,,,,,,,

In [ ]:
#Loading groundTruth
groundTruthDf = pd.read_excel ('GroundTruth.xlsx', sheet_name='Sheet1')
groundTruthDf = groundTruthDf.fillna('')
display(groundTruthDf)

,Index,Prediction Question (i),Image Index (j),Image Type,Predict,Best Acceptable Reason,All Acceptable Reasons,All Reasons,Best Acceptable Fix,All Acceptable Fixes,All Fixes
0,1,1.0,1.0,L,"1,3",4.0,"1,4","1,2,3,4,5,6",3.0,"1,3,5","1,2,3,4,5,6,7"
1,2,1.0,2.0,M,"1,3",1.0,1.0,"1,2,3,4,5,6",1.0,1.0,"1,2,3,4,5,6,7"
2,3,1.0,3.0,L,"1,3",5.0,"1,5","1,2,3,4,5,6",6.0,"1,6","1,2,3,4,5,6,7"
3,4,1.0,4.0,U,"1,3",,,"1,2,3,4,5,6",,,"1,2,3,4,5,6,7"
4,5,1.0,5.0,M,"1,3",5.0,"1,5","1,2,3,4,5,6",6.0,"1,6","1,2,3,4,5,6,7"
5,6,2.0,1.0,L,1.0,2.0,"1,2","1,2,3,4,5,6",2.0,"1,2","1,2,3,4,5,6,7"
6,7,2.0,2.0,U,1.0,,,"1,2,3,4,5,6",,,"1,2,3,4,5,6,7"
7,8,2.0,3.0,M,1.0,1.0,1.0,"1,2,3,4,5,6",1.0,1.0,"1,2,3,4,5,6,7"
8,9,2.0,4.0,M,1.0,1.0,1.0,"1,2,3,4,5,6",1.0,1.0,"1,2,3,4,5,6,7"
9,10,2.0,5.0,M,1.0,1.0,"1,5","1,2,3,4,5,6",1.0,1.0,"1,2,3,4,5,6,7"


In [ ]:
# #store in-person participants' responses in a separate dataframe
# In_person_df= all_dfs[all_dfs.ParticipantID.str.startswith('I')]
# In_person_df= In_person_df.fillna('')
# #display(In_person_df)
# #print(In_person_df.shape[0])

# #store online participants' responses in a separate dataframe
# Online_df= all_dfs[all_dfs.ParticipantID.str.startswith('O')]
# Online_df= Online_df.fillna('')
# #display(Online_df)

In [ ]:
#GroundTruth for the entry survey
groundTruthDf['Prediction Question (i)'] = groundTruthDf['Prediction Question (i)'].astype(int).astype(str)
groundTruthDf['Image Index (j)'] = groundTruthDf['Image Index (j)'].astype(int).astype(str)

#Function to convert all columns into string

In [ ]:
def change_type(row_val):

  if type(row_val)==float:
    row_val = str(int(row_val))

  elif type(row_val)==str:
    row_val = row_val

  return row_val

Add second best reason and second best fix to the data frame

In [ ]:
def sym_diff(col_a, col_b):

  symmetric_diff =  set(col_a).symmetric_difference(set(col_b))

  numeric_vals = [value for value in symmetric_diff if value.isnumeric()]

  numeric_str = ",".join(numeric_vals)

  return numeric_str

In [ ]:
#Groundtruth
groundTruthDf['Best Acceptable Reason'] = groundTruthDf['Best Acceptable Reason'].replace("",0.0)

groundTruthDf['Best Acceptable Fix'] = groundTruthDf['Best Acceptable Fix'].replace("",0.0)

groundTruthDf['All Acceptable Fixes'] = groundTruthDf['All Acceptable Fixes'].replace("",0.0)

groundTruthDf['All Acceptable Reasons'] = groundTruthDf['All Acceptable Reasons'].replace("",0.0)



In [ ]:
#Groundtruth

groundTruthDf['Best Acceptable Reason'] = groundTruthDf['Best Acceptable Reason'].apply(lambda x: change_type(x))
groundTruthDf['All Acceptable Reasons'] = groundTruthDf['All Acceptable Reasons'].apply(lambda x: change_type(x))
groundTruthDf['All Reasons'] = groundTruthDf['All Reasons'].apply(lambda x: change_type(x))


groundTruthDf['Best Acceptable Fix'] = groundTruthDf['Best Acceptable Fix'].apply(lambda x: change_type(x))
groundTruthDf['All Acceptable Fixes'] = groundTruthDf['All Acceptable Fixes'].apply(lambda x: change_type(x))
groundTruthDf['All Fixes'] = groundTruthDf['All Fixes'].apply(lambda x: change_type(x))



In [ ]:
#Groundtruth: creating the "second best reason" column
groundTruthDf['Second Best Reason'] = groundTruthDf.apply(lambda x: sym_diff(x['Best Acceptable Reason'], x['All Acceptable Reasons']), axis=1)
groundTruthDf['Second Best Fix'] = groundTruthDf.apply(lambda x: sym_diff(x['Best Acceptable Fix'], x['All Acceptable Fixes']), axis=1)
#display(groundTruth_Entry_Df)


In [ ]:
#Groundtruth: creating the wrong reasons column
groundTruthDf['Wrong Reasons'] = groundTruthDf.apply(lambda x: sym_diff(x['All Acceptable Reasons'], x['All Reasons']), axis=1)
groundTruthDf['Wrong Fixes'] = groundTruthDf.apply(lambda x: sym_diff(x['All Acceptable Fixes'], x['All Fixes']), axis=1)
display(groundTruthDf)

,Index,Prediction Question (i),Image Index (j),Image Type,Predict,Best Acceptable Reason,All Acceptable Reasons,All Reasons,Best Acceptable Fix,All Acceptable Fixes,All Fixes,Second Best Reason,Second Best Fix,Wrong Reasons,Wrong Fixes
0,1,1,1,L,"1,3",4,"1,4","1,2,3,4,5,6",3,"1,3,5","1,2,3,4,5,6,7",1,"1,5","3,5,2,6","7,4,2,6"
1,2,1,2,M,"1,3",1,1,"1,2,3,4,5,6",1,1,"1,2,3,4,5,6,7",,,"3,5,2,6,4","7,4,3,5,2,6"
2,3,1,3,L,"1,3",5,"1,5","1,2,3,4,5,6",6,"1,6","1,2,3,4,5,6,7",1,1,"3,2,6,4","7,4,3,5,2"
3,4,1,4,U,"1,3",0,0,"1,2,3,4,5,6",0,0,"1,2,3,4,5,6,7",,,"3,5,1,0,2,6,4","1,7,0,4,3,5,2,6"
4,5,1,5,M,"1,3",5,"1,5","1,2,3,4,5,6",6,"1,6","1,2,3,4,5,6,7",1,1,"3,2,6,4","7,4,3,5,2"
5,6,2,1,L,1.0,2,"1,2","1,2,3,4,5,6",2,"1,2","1,2,3,4,5,6,7",1,1,"3,5,6,4","7,4,3,5,6"
6,7,2,2,U,1.0,0,0,"1,2,3,4,5,6",0,0,"1,2,3,4,5,6,7",,,"3,5,1,0,2,6,4","1,7,0,4,3,5,2,6"
7,8,2,3,M,1.0,1,1,"1,2,3,4,5,6",1,1,"1,2,3,4,5,6,7",,,"3,5,2,6,4","7,4,3,5,2,6"
8,9,2,4,M,1.0,1,1,"1,2,3,4,5,6",1,1,"1,2,3,4,5,6,7",,,"3,5,2,6,4","7,4,3,5,2,6"
9,10,2,5,M,1.0,1,"1,5","1,2,3,4,5,6",1,1,"1,2,3,4,5,6,7",5,,"3,2,6,4","7,4,3,5,2,6"


In [ ]:
#storing the entry groundTruth as a dictionary
groundTruth= groundTruthDf.to_dict(orient = 'records')
predict_truth = 'Predict'
imageType_truth = 'Image Type'
reason_truth = 'All Acceptable Reasons'
best_reason_truth = 'Best Acceptable Reason'
second_best_reason_truth = 'Second Best Reason'
wrong_reason_truth = 'Wrong Reasons'
fix_truth = 'All Acceptable Fixes'
best_fix_truth = 'Best Acceptable Fix'
second_best_fix_truth = 'Second Best Fix'
wrong_fix_truth = 'Wrong Fixes'

#print(groundTruth)

In [ ]:
columns_df_Entry = df_Entry.columns
display(columns_df_Entry)
display(df_Entry.index)



Index(['ParticipantID', 'Entery_1_Predict', 'Entery_1_Reason_Opt1',
       'Entery_1_Reason_Opt1_6_TEXT', 'Entery_1_Reason_Opt2',
       'Entery_1_Reason_Opt2_6_TEXT', 'Entery_1_Reason_Opt3',
       'Entery_1_Reason_Opt3_6_TEXT', 'Entery_1_Reason_Opt4',
       'Entery_1_Reason_Opt4_6_TEXT',
       ...
       'Entery_5_Fix_Opt1', 'Entery_5_Fix_Opt1_7_TEXT', 'Entery_5_Fix_Opt2',
       'Entery_5_Fix_Opt2_7_TEXT', 'Entery_5_Fix_Opt3',
       'Entery_5_Fix_Opt3_7_TEXT', 'Entery_5_Fix_Opt4',
       'Entery_5_Fix_Opt4_7_TEXT', 'Entery_5_Fix_Opt5',
       'Entery_5_Fix_Opt5_7_TEXT'],
      dtype='object', length=106)

RangeIndex(start=0, stop=11, step=1)

In [ ]:
#Set index on the df_Entry
df_Entry = df_Entry.set_index(columns_df_Entry[0])

columns_df_Entry = df_Entry.columns
display(columns_df_Entry)
display(df_Entry.index)

# #Set index on the df_Exit
# df_Exit = df_Exit.set_index(columns_df_Exit[0])

# columns_df_Exit = df_Exit.columns
# display(columns_df_Exit)
# display(df_Exit.index)


Index(['Entery_1_Predict', 'Entery_1_Reason_Opt1',
       'Entery_1_Reason_Opt1_6_TEXT', 'Entery_1_Reason_Opt2',
       'Entery_1_Reason_Opt2_6_TEXT', 'Entery_1_Reason_Opt3',
       'Entery_1_Reason_Opt3_6_TEXT', 'Entery_1_Reason_Opt4',
       'Entery_1_Reason_Opt4_6_TEXT', 'Entery_1_Reason_Opt5',
       ...
       'Entery_5_Fix_Opt1', 'Entery_5_Fix_Opt1_7_TEXT', 'Entery_5_Fix_Opt2',
       'Entery_5_Fix_Opt2_7_TEXT', 'Entery_5_Fix_Opt3',
       'Entery_5_Fix_Opt3_7_TEXT', 'Entery_5_Fix_Opt4',
       'Entery_5_Fix_Opt4_7_TEXT', 'Entery_5_Fix_Opt5',
       'Entery_5_Fix_Opt5_7_TEXT'],
      dtype='object', length=105)

Index(['O_P9', 'O_P13', 'O_P14', 'O_P17', 'O_P18', 'I_P12', 'I_P13', 'I_P14',
       'O_P19', 'O_P20', 'I_P17'],
      dtype='object', name='ParticipantID')

In [ ]:
# Get this data from participants' responses
nImages = 5
nLoops = 5
nParticipants = 11

imageType_key = "imageType"
predictionScores_key = "predictionScores"
reasonScores_key = "reasonScores"
fixScores_key = "fixScores"
totalPRFScores_key = "totalPRFScores"
finalResult_key = "finalResult"
selected_key = "selected"

# This will a list of dictionaries, one for each image. for e.g. [1 x 25]
'''
# memory allocation this way will make copies of the reference to the dicionary instead of making copies of the actual dictionaries
# i don't kinda understand why but I don't like it lol
d = dict()
d[imageType_key] = ""
d[selected_key] = False
d[predictionScores_key] = [0] * nParticipants
d[reasonScores_key] = [0] * nParticipants
d[fixScores_key] = [0] * nParticipants
d[totalPRFScores_key] = [0] * nParticipants
d[finalResult_key] = [0] * nParticipants
results = [d] * nImages * nLoops
'''
# create results buffer
results = []
# create new objects for each individual image
for _ in range(nImages * nLoops):
  d = dict()
  d[imageType_key] = ""
  d[selected_key] = False
  d[predictionScores_key] = [0] * nParticipants
  d[reasonScores_key] = [0] * nParticipants
  d[fixScores_key] = [0] * nParticipants
  d[totalPRFScores_key] = [0] * nParticipants
  d[finalResult_key] = [0] * nParticipants
  results.append(d)

# Reason scores
reasonScore_1 = 0.33
reasonScore_2 = 0.22

# Fix scores
fixScore_1 = 0.33
fixScore_2 = 0.22

# Image Types
mightFail = "M"
unlikelyFail = "U"
likelyFail = "L"

In [ ]:
def parseIntoSortedList(inString, delimiter = ','):
  splitList = inString.split(delimiter)
  for i in range(0, len(splitList)):
    splitList[i] = int(float(splitList[i]))
  splitList.sort()
  return splitList

In [ ]:
def does_fix_match_reasons(f, r_list):
  mapping = ['1', '2', '4', '3,5', '6', '7']
  f_list = [mapping[x - 1].split(',') for x in r_list]
  f_list = [int(num) for sublist in f_list for num in sublist]
  return f in f_list


# Entry survey for each participants answers
participantIndex = 0
for row in df_Entry.itertuples(index = False):
  # "decompress" participant row data
  dictRow = row._asdict()
  dictRow = dict(zip(columns_df_Entry, list(dictRow.values())))

  # for each loop of question images
  for loopIndex in range(1, nLoops+1):
    #Entery_1_Predict
    # Parse the participant's prediction into a sorted list of numbers.
    predictionAttribute = "Entery_" + str(loopIndex) + "_Predict"
    predictionString = dictRow[predictionAttribute]
    prediction = parseIntoSortedList(predictionString, ",")

    # for each question image
    for index in range(1,nImages + 1):
        
      # set initial scores
      pScore = 0.0
      rScore = 0.0
      fScore = 0.0

      # was image 'index' selected
      selected = index in prediction

      # get numloops x imagesperloop index
      imageIndex = ((loopIndex - 1)*nLoops) + index - 1
      # get image data and image type
      predictionTruth = groundTruth[imageIndex]
      imageTypeTruth = predictionTruth[imageType_truth]

      # image is unlikely to fail but selected, failure scores
      if imageTypeTruth == unlikelyFail and selected == True:
        pScore = 0.0
        rScore = 0.0
        fScore = 0.0
      # image is unlikely to and not selected, perfect prediction score
      elif imageTypeTruth == unlikelyFail and selected == False:
        pScore = 1.0
        rScore = 0.0
        fScore = 0.0
      # image is likley to fail but not selected, failure scores
      elif imageTypeTruth == likelyFail and selected == False:
        pScore = 0.0
        rScore = 0.0
        fScore = 0.0
      # image is might fail but not selected, treat as unlikely, perfect prediction score
      elif imageTypeTruth == mightFail and selected == False:
        pScore = 1.0
        rScore = 0.0
        fScore = 0.0
      # image is might or likely to fail and selected, more compilcated scoring
      elif (imageTypeTruth == mightFail or imageTypeTruth == likelyFail) and selected == True:
        # correct prediction score
        pScore = 0.34

        # parse the participant's reason into a sorted list of numbers. 
        #Entery_1_Reason_Opt1
        reasonAttribute = "Entery_" + str(loopIndex) + "_Reason_Opt" + str(index)
        reasonString = str(dictRow[reasonAttribute])
        reason = parseIntoSortedList(reasonString, ",")
        #print(reasonAttribute)

        # parse the participant's fix into a number. 
        #Entery_1_Fix_Opt1
        fixAttribute = "Entery_" + str(loopIndex) + "_Fix_Opt" + str(index)  
        fixString = dictRow[fixAttribute]
        fix = int(fixString)
        
        # get "best reason", "second best reason", and "wrong reason" for image
        bestReasonTruthString = predictionTruth[best_reason_truth]
        # bestReasonTruth = parseIntoSortedList(bestReasonTruthString, ",")
        bestReasonTruth = int(bestReasonTruthString)
        secondBestReasonTruth = [int(x) for x in predictionTruth[second_best_reason_truth].split(',') if x is not '']
        # if participantIndex ==13 and loopIndex==1:
        #  secondBestReasonTruth = secondBestReasonTruth[0]
        wrongReasonTruth = predictionTruth[wrong_reason_truth]

        # get "best fix", "second best fixes", and "wrong fixes" for image
        bestFixTruth = predictionTruth[best_fix_truth]
        secondBestFixTruth = [int(x) for x in predictionTruth[second_best_fix_truth].split(',') if x is not '']
        wrongFixTruth = [int(x) for x in predictionTruth[wrong_fix_truth].split(',') if x is not '']
        fixTruthString = str(predictionTruth[fix_truth])
        fixTruth = parseIntoSortedList(fixTruthString, ",")
        # if participantIndex ==13 and loopIndex==1:
        #   print(str(secondBestReasonTruth))
        #participant reason 2,6
        #best reason 1
        #second best 2,4,6
        #wrong reason 3,5 
        #0.34 + (0.33)
        # if participantIndex ==13 and loopIndex==1:
        #   print(str(imageIndex) + " " +str(reason) + " " + str(bestReasonTruth) + " " + str(secondBestReasonTruth) + " " + str(wrongReasonTruth)+ " " +str(reasonAttribute))


        # the best reason was selected
        if (bestReasonTruth in reason):
          # calculate r-score
          rScore = reasonScore_1 - (0.05 * len([x for x in reason if x in parseIntoSortedList(wrongReasonTruth, ",")]))
          if participantIndex ==13 and loopIndex==1:
           print(str("Hello")) 

          # selected fix matches reason
          if does_fix_match_reasons(fix, [bestReasonTruth]):
            fScore = fixScore_1
          # selected fix is acceptable
          elif fix in fixTruth:
            fScore = fixScore_2
          # selected wrong fix option
          else:
            fScore = 0.0
        # participant has picked the "second best reason" and not the "best reason"
        #elif (secondBestReasonTruth in reason): #and (bestReasonTruth not in reason): 
        else:
          rScore = 0
          nsecondBestReasons = len([x for x in reason if x in secondBestReasonTruth])
          if nsecondBestReasons > 0: 
            rScore = reasonScore_1 - (0.1 * len([x for x in reason if x in parseIntoSortedList(wrongReasonTruth, ",")]))
        
            
            # selected fix matches reason
            if does_fix_match_reasons(fix, reason):
              fScore = fixScore_1
            # selected fix is acceptable
            elif fix in fixTruth:
              fScore = fixScore_2
            # selected wrong fix option
            else:
              fScore = 0.0

        # participant has picked the wrong reasons
          else:
            # selected best fix option
            if bestFixTruth == fix:
              fScore = fixScore_1
            # selected second best/acceptable fix option
            elif fix in fixTruth:
              fScore = fixScore_2
            # selected wrong fix option
            else:
              fScore = 0.0
        # should not happen
      else:
        print("ERROR, unhandled image type and selection value:", imageTypeTruth, selected)
      #print(participantIndex, imageIndex, loopIndex, index, selected, imageTypeTruth, pScore, rScore, fScore) 

      d = results[imageIndex]
      d[imageType_key] = imageTypeTruth
      d[selected_key] = selected

      d[predictionScores_key][participantIndex] = pScore 
      d[reasonScores_key][participantIndex] = rScore 
      d[fixScores_key][participantIndex] = fScore 
      d[totalPRFScores_key][participantIndex] = pScore + rScore + fScore

  # go to next participant
  participantIndex += 1
    
  #if participantIndex >= 3: break
print(results[4]) #image loop 1 image 5

{'imageType': 'M', 'selected': False, 'predictionScores': [1.0, 0.34, 1.0, 1.0, 1.0, 1.0, 0.34, 0.34, 1.0, 1.0, 1.0], 'reasonScores': [0.0, 0, 0.0, 0.0, 0.0, 0.0, 0.23, 0.23, 0.0, 0.0, 0.0], 'fixScores': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.33, 0.33, 0.0, 0.0, 0.0], 'totalPRFScores': [1.0, 0.34, 1.0, 1.0, 1.0, 1.0, 0.9000000000000001, 0.9000000000000001, 1.0, 1.0, 1.0], 'finalResult': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}


In [ ]:
q_index = 1
for dictionary in results: 
  a = dictionary[totalPRFScores_key]
  print(q_index, ["{0:0.2f}".format(i) for i in a])
  q_index += 1

1 ['0.56', '1.00', '0.95', '1.00', '1.00', '0.95', '0.00', '1.00', '1.00', '1.00', '1.00']
2 ['1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00']
3 ['1.00', '1.00', '0.95', '1.00', '1.00', '0.00', '0.67', '0.95', '1.00', '1.00', '0.00']
4 ['1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00']
5 ['1.00', '0.34', '1.00', '1.00', '1.00', '1.00', '0.90', '0.90', '1.00', '1.00', '1.00']
6 ['1.00', '1.00', '0.95', '1.00', '1.00', '1.00', '1.00', '1.00', '0.95', '1.00', '1.00']
7 ['1.00', '1.00', '0.00', '1.00', '1.00', '1.00', '0.00', '1.00', '1.00', '1.00', '1.00']
8 ['1.00', '1.00', '0.95', '1.00', '1.00', '1.00', '1.00', '0.67', '1.00', '1.00', '1.00']
9 ['1.00', '0.34', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00']
10 ['1.00', '0.34', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00', '1.00']
11 ['1.00', '1.00', '1.00', '1.00', '0.95', '0.67', '1.00', '1.00', '1.00', '1.00', '1.00

In [ ]:
# df:  Final data frame with the results per participants


df_en = pd.DataFrame(columns=['ParticipantID'])
df_en['ParticipantID'] = df_Entry.index

index = 0
for dictionary in results:
  index += 1
  PRFScore = dictionary[totalPRFScores_key]
  #print(PRFScore)
  colName = "PRF_Score" + str(index)
  df_en[colName] = PRFScore
dictionary[totalPRFScores_key]

df_en.columns = ['ParticipantID',
              'L1_Ent_Obstruction1_LF','L1_Ent_Success1_MF','L1_Ent_TooFar_LF','L1_Ent_Success2_UF','L1_Ent_Success3_MF',
              'L2_Ent_Angle1_LF','L2_Ent_Success1_UF','L2_Ent_TooBright_MF', 'L2_Ent_Success2_MF', 'L2_Ent_Success3_MF',
              'L3_Ent_Clutter1_MF','L3_Ent_Success1_MF','L3_Ent_Obstruction2_LF','L3_Ent_Success2_UF','L3_Ent_Success3_MF',
              'L4_Ent_TooClose_LF','L4_Ent_Success1_MF','L4_Ent_Clutter2_MF', 'L4_Ent_Success2_MF', 'L4_Ent_Success3_MF',
              'L5_Ent_TooDark_LF','L5_Ent_Success1_UF','L5_Ent_Angle2_LF','L5_Ent_Success2_MF','L5_Ent_Success3_UF']

display(df_en)




,ParticipantID,L1_Ent_Obstruction1_LF,L1_Ent_Success1_MF,L1_Ent_TooFar_LF,L1_Ent_Success2_UF,L1_Ent_Success3_MF,L2_Ent_Angle1_LF,L2_Ent_Success1_UF,L2_Ent_TooBright_MF,L2_Ent_Success2_MF,L2_Ent_Success3_MF,L3_Ent_Clutter1_MF,L3_Ent_Success1_MF,L3_Ent_Obstruction2_LF,L3_Ent_Success2_UF,L3_Ent_Success3_MF,L4_Ent_TooClose_LF,L4_Ent_Success1_MF,L4_Ent_Clutter2_MF,L4_Ent_Success2_MF,L4_Ent_Success3_MF,L5_Ent_TooDark_LF,L5_Ent_Success1_UF,L5_Ent_Angle2_LF,L5_Ent_Success2_MF,L5_Ent_Success3_UF
0,O_P9,0.56,1.0,1.00,1.0,1.00,1.00,1.0,1.00,1.00,1.00,1.00,1.0,1.00,1.0,1.00,0.00,1.0,1.00,1.0,1.00,1.0,1.0,0.67,1.0,1.0
1,O_P13,1.00,1.0,1.00,1.0,0.34,1.00,1.0,1.00,0.34,0.34,1.00,1.0,1.00,0.0,0.34,1.00,1.0,1.00,1.0,0.34,1.0,1.0,1.00,1.0,1.0
2,O_P14,0.95,1.0,0.95,1.0,1.00,0.95,0.0,0.95,1.00,1.00,1.00,1.0,0.89,0.0,1.00,0.95,1.0,1.00,1.0,1.00,1.0,1.0,0.95,1.0,0.0
3,O_P17,1.00,1.0,1.00,1.0,1.00,1.00,1.0,1.00,1.00,1.00,1.00,1.0,1.00,1.0,1.00,0.95,1.0,1.00,1.0,1.00,1.0,1.0,1.00,1.0,1.0
4,O_P18,1.00,1.0,1.00,1.0,1.00,1.00,1.0,1.00,1.00,1.00,0.95,1.0,0.67,1.0,1.00,0.00,1.0,0.67,1.0,1.00,1.0,1.0,1.00,1.0,1.0
5,I_P12,0.95,1.0,0.00,1.0,1.00,1.00,1.0,1.00,1.00,1.00,0.67,1.0,0.95,1.0,1.00,1.00,1.0,1.00,1.0,1.00,1.0,1.0,0.95,1.0,1.0
6,I_P13,0.00,1.0,0.67,1.0,0.90,1.00,0.0,1.00,1.00,1.00,1.00,1.0,0.67,0.0,1.00,0.56,1.0,1.00,1.0,0.57,1.0,1.0,1.00,1.0,0.0
7,I_P14,1.00,1.0,0.95,1.0,0.90,1.00,1.0,0.67,1.00,1.00,1.00,1.0,1.00,1.0,1.00,1.00,1.0,1.00,1.0,0.67,1.0,1.0,1.00,1.0,1.0
8,O_P19,1.00,1.0,1.00,1.0,1.00,0.95,1.0,1.00,1.00,1.00,1.00,1.0,1.00,1.0,1.00,1.00,1.0,1.00,1.0,1.00,1.0,1.0,0.84,1.0,1.0
9,O_P20,1.00,1.0,1.00,1.0,1.00,1.00,1.0,1.00,1.00,1.00,1.00,1.0,1.00,1.0,1.00,1.00,1.0,1.00,1.0,1.00,0.0,1.0,1.00,1.0,1.0


In [ ]:
df_en = df_en.assign(avg=df_en.mean(axis=1))
display(df_en)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  """Entry point for launching an IPython kernel.


,ParticipantID,L1_Ent_Obstruction1_LF,L1_Ent_Success1_MF,L1_Ent_TooFar_LF,L1_Ent_Success2_UF,L1_Ent_Success3_MF,L2_Ent_Angle1_LF,L2_Ent_Success1_UF,L2_Ent_TooBright_MF,L2_Ent_Success2_MF,L2_Ent_Success3_MF,L3_Ent_Clutter1_MF,L3_Ent_Success1_MF,L3_Ent_Obstruction2_LF,L3_Ent_Success2_UF,L3_Ent_Success3_MF,L4_Ent_TooClose_LF,L4_Ent_Success1_MF,L4_Ent_Clutter2_MF,L4_Ent_Success2_MF,L4_Ent_Success3_MF,L5_Ent_TooDark_LF,L5_Ent_Success1_UF,L5_Ent_Angle2_LF,L5_Ent_Success2_MF,L5_Ent_Success3_UF,avg
0,O_P9,0.56,1.0,1.00,1.0,1.00,1.00,1.0,1.00,1.00,1.00,1.00,1.0,1.00,1.0,1.00,0.00,1.0,1.00,1.0,1.00,1.0,1.0,0.67,1.0,1.0,0.9292
1,O_P13,1.00,1.0,1.00,1.0,0.34,1.00,1.0,1.00,0.34,0.34,1.00,1.0,1.00,0.0,0.34,1.00,1.0,1.00,1.0,0.34,1.0,1.0,1.00,1.0,1.0,0.8280
2,O_P14,0.95,1.0,0.95,1.0,1.00,0.95,0.0,0.95,1.00,1.00,1.00,1.0,0.89,0.0,1.00,0.95,1.0,1.00,1.0,1.00,1.0,1.0,0.95,1.0,0.0,0.8636
3,O_P17,1.00,1.0,1.00,1.0,1.00,1.00,1.0,1.00,1.00,1.00,1.00,1.0,1.00,1.0,1.00,0.95,1.0,1.00,1.0,1.00,1.0,1.0,1.00,1.0,1.0,0.9980
4,O_P18,1.00,1.0,1.00,1.0,1.00,1.00,1.0,1.00,1.00,1.00,0.95,1.0,0.67,1.0,1.00,0.00,1.0,0.67,1.0,1.00,1.0,1.0,1.00,1.0,1.0,0.9316
5,I_P12,0.95,1.0,0.00,1.0,1.00,1.00,1.0,1.00,1.00,1.00,0.67,1.0,0.95,1.0,1.00,1.00,1.0,1.00,1.0,1.00,1.0,1.0,0.95,1.0,1.0,0.9408
6,I_P13,0.00,1.0,0.67,1.0,0.90,1.00,0.0,1.00,1.00,1.00,1.00,1.0,0.67,0.0,1.00,0.56,1.0,1.00,1.0,0.57,1.0,1.0,1.00,1.0,0.0,0.7748
7,I_P14,1.00,1.0,0.95,1.0,0.90,1.00,1.0,0.67,1.00,1.00,1.00,1.0,1.00,1.0,1.00,1.00,1.0,1.00,1.0,0.67,1.0,1.0,1.00,1.0,1.0,0.9676
8,O_P19,1.00,1.0,1.00,1.0,1.00,0.95,1.0,1.00,1.00,1.00,1.00,1.0,1.00,1.0,1.00,1.00,1.0,1.00,1.0,1.00,1.0,1.0,0.84,1.0,1.0,0.9916
9,O_P20,1.00,1.0,1.00,1.0,1.00,1.00,1.0,1.00,1.00,1.00,1.00,1.0,1.00,1.0,1.00,1.00,1.0,1.00,1.0,1.00,0.0,1.0,1.00,1.0,1.0,0.9600
